In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#import findspark
#findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Fashion-Recommender-Engineer")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "8")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

26/03/02 01:14:23 WARN Utils: Your hostname, MacBook-Air-M4.local resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
26/03/02 01:14:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/03/02 01:14:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/03/02 01:14:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Verify spark
spark

## Load CSV dataset into Spark DataFrame

In [ ]:
CSV_PATH = "fashion_dataset_fix.csv"

df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(CSV_PATH)
)
df.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- product_code: integer (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- product_type_no: integer (nullable = true)
 |-- product_type_name: string (nullable = true)
 |-- product_group_name: string (nullable = true)
 |-- graphical_appearance_no: integer (nullable = true)
 |-- graphical_appearance_name: string (nullable = true)
 |-- colour_group_code: integer (nullable = true)
 |-- colour_group_name: string (nullable = true)
 |-- perceived_colour_value_id: integer (nullable = true)
 |-- perceived_colour_value_name: string (nullable = true)
 |-- perceived_colour_master_id: integer (nullable = true)
 |-- perceived_colour_master_name: string (nullable = true)
 |-- department_no: integer (nullable = true)
 |-- department_name: string (nullable = true)
 |-- index_code: string (nullable = true)
 |-- index_name: string (nullable = true)
 |-- index_group_no: integer (nullable = true)
 |-- index_group_name: string (nullable = true)

## Convert Spark DataFrame → Pandas DataFrame

In [5]:
df = df.drop("_c0")

# Select & reorder columns
columns = [
    "item_id",
    "category",
    "subcategory",
    "sleeve_type",
    "season",
    "fabric",
    "occasion",
    "formality_level",
    "size_range",
    "brand",
    "view_count",
    "click_count",
    "purchase_count",
    "length_cm",
    "price",
    "stocks",
]

df = df.select(columns)

In [6]:
# Convert Spark -> Dataframe
df_cleaned = df.toPandas()
print(type(df_cleaned))
df_cleaned

<class 'pandas.core.frame.DataFrame'>


,item_id,category,subcategory,sleeve_type,season,fabric,occasion,formality_level,size_range,brand,view_count,click_count,purchase_count,length_cm,price,stocks
0,TNC_000001,bottoms,men_cargos,long,winter,polyester,casual,high,S,Tommy Hilfiger,93,192,4,97,262476,35
1,TNC_000002,tops,formal_shirts,long,winter,linen,party,medium,S,HnM,2941,84,5,77,266987,182
2,TNC_000003,tops,formal_shirts,long,winter,denim,party,high,XL,HnM,401,434,35,71,286065,32
3,TNC_000004,tops,printed_tshirts,short,all-season,linen,office,low,XXL,Tommy Hilfiger,318,85,101,70,153376,149
4,TNC_000005,tops,formal_shirts,long,all-season,polyester,casual,low,L,ZARA,857,153,67,69,323155,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099995,TNC_1099996,tops,solid_tshirts,short,summer,polyester,party,medium,M,HnM,2924,183,2,73,140786,163
1099996,TNC_1099997,bottoms,formal_pants,long,summer,denim,party,high,XXL,Polo,2362,94,32,95,353863,198
1099997,TNC_1099998,bottoms,men_cargos,long,winter,fleece,casual,low,M,Adidas,617,147,39,97,299627,78
1099998,TNC_1099999,tops,solid_tshirts,short,summer,cotton,office,low,XXL,Tommy Hilfiger,326,0,25,71,153123,172


## Insert Spark DataFrame into a Table

In [7]:
spark_sql = (
    SparkSession.builder
    .appName("fashion-recommender")
    .enableHiveSupport()
    .getOrCreate()
)
df.write.mode("overwrite").saveAsTable("fashion_sql_datasets")

26/02/10 23:56:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
spark.sql("SELECT * FROM fashion_sql_datasets").show()

NameError: name 'spark_sql' is not defined

In [9]:
df.write.mode("overwrite").parquet("hdfs/fashion_dataset")